# Internações Respiratórias RJ — Pipeline e Análise Exploratória (SUS 2012–2024)

Este notebook consolida as **internações por doenças respiratórias** (SUS) no **município do Rio de Janeiro** e prepara a **série diária de internações** para modelagem.
O foco é: leitura dos dados brutos anuais, filtros clínicos e geográficos coerentes com exposição ambiental e uma **análise exploratória inicial**.

> **Escopo**: 2012–2024, diagnósticos respiratórios (CID-10 selecionados), residentes do Rio (IBGE 330455) e permanência hospitalar > 0.

## Objetivos desta etapa
1. **Carregar** todos os CSVs anuais (2012–2024) diretamente do GitHub (*raw*).
2. **Padronizar** campos-chave (datas, CID-10) e **filtrar** registros de interesse.
3. **Construir** a série **diária** de internações (`data_dia` × `internacoes`).
4. **Explorar** o comportamento da série (tendência, sazonalidade, distribuição) e variáveis-chave (idade, permanência).
5. **Salvar** a série diária para uso posterior na modelagem.

## Configurações e importações
Bibliotecas usadas e parâmetros gerais.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
from statsmodels.tsa.seasonal import STL
from statsmodels.tsa.stattools import acf
from scipy.signal import periodogram
from numpy.linalg import lstsq
from __future__ import annotations
import numpy as np
import pandas as pd

pd.set_option("display.max_columns", 100)
pd.set_option("display.width", 120)
plt.rcParams["figure.figsize"] = (12, 5)
plt.rcParams["axes.grid"] = True

## Carregamento e Unificação das Estações

Nesta etapa:

- **Origem:** `data/datasus/Ano/dados_filtrados_{ANO}.csv` (2012–2024) no branch `refs/heads/main`.
- **Concatenação:** une todos os anos com `ignore_index=True`.

In [4]:
anos = range(2012, 2025)
tpl = "https://raw.githubusercontent.com/EIC-BCC/25_2-QualiAr/refs/heads/main/data/DataSus/Por-Ano/dados_filtrados_{ano}.csv"

dfs = []
for ano in anos:
    url = tpl.format(ano=ano)
    print(f"Lendo: {url}")
    df = pd.read_csv(url, encoding="utf-8")
    df["ano_arquivo"] = ano
    dfs.append(df)

df_sus = pd.concat(dfs, ignore_index=True)

Lendo: https://raw.githubusercontent.com/EIC-BCC/25_2-QualiAr/refs/heads/main/data/DataSus/Por-Ano/dados_filtrados_2012.csv
Lendo: https://raw.githubusercontent.com/EIC-BCC/25_2-QualiAr/refs/heads/main/data/DataSus/Por-Ano/dados_filtrados_2013.csv
Lendo: https://raw.githubusercontent.com/EIC-BCC/25_2-QualiAr/refs/heads/main/data/DataSus/Por-Ano/dados_filtrados_2014.csv
Lendo: https://raw.githubusercontent.com/EIC-BCC/25_2-QualiAr/refs/heads/main/data/DataSus/Por-Ano/dados_filtrados_2015.csv


C:\Users\jhter\AppData\Local\Temp\ipykernel_2252\237882505.py:8: DtypeWarning: Columns (11,18) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(url, encoding="utf-8")


Lendo: https://raw.githubusercontent.com/EIC-BCC/25_2-QualiAr/refs/heads/main/data/DataSus/Por-Ano/dados_filtrados_2016.csv


C:\Users\jhter\AppData\Local\Temp\ipykernel_2252\237882505.py:8: DtypeWarning: Columns (11,18) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(url, encoding="utf-8")


Lendo: https://raw.githubusercontent.com/EIC-BCC/25_2-QualiAr/refs/heads/main/data/DataSus/Por-Ano/dados_filtrados_2017.csv


...

C:\Users\jhter\AppData\Local\Temp\ipykernel_2252\237882505.py:8: DtypeWarning: Columns (11,18) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(url, encoding="utf-8")


Lendo: https://raw.githubusercontent.com/EIC-BCC/25_2-QualiAr/refs/heads/main/data/DataSus/Por-Ano/dados_filtrados_2023.csv


C:\Users\jhter\AppData\Local\Temp\ipykernel_2252\237882505.py:8: DtypeWarning: Columns (11,18) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(url, encoding="utf-8")


Lendo: https://raw.githubusercontent.com/EIC-BCC/25_2-QualiAr/refs/heads/main/data/DataSus/Por-Ano/dados_filtrados_2024.csv


C:\Users\jhter\AppData\Local\Temp\ipykernel_2252\237882505.py:8: DtypeWarning: Columns (11,18) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(url, encoding="utf-8")


## Inspeção inicial

Visão geral de dimensões, tipos, amostra e dados ausentes. A coluna de data principal
virá de `DT_INTER` (quando numérica tipo `YYYYMMDD`) ou de `data_formatada` (se já estiver disponível).

In [5]:
print("Dimensões:", df_sus.shape)

print("\nTipos:")
print(df_sus.dtypes)

print("\nAmostra:")
display(df_sus.head(10))

print("\nValores ausentes por coluna (top 20):")
na_counts = df_sus.isna().sum().sort_values(ascending=False)
na_perc   = (df_sus.isna().mean()*100).round(2).sort_values(ascending=False)
display(pd.DataFrame({"NA_count": na_counts, "NA_%": na_perc}).head(20))

Dimensões: (5002306, 21)

Tipos:
UF_ZI              int64
ANO_CMPT           int64
MES_CMPT           int64
MUNIC_RES          int64
MUNIC_MOV          int64
CEP                int64
NASC               int64
SEXO               int64
DT_INTER           int64
DT_SAIDA           int64
DIAG_PRINC        object
DIAG_SECUN        object
IDADE              int64
COD_IDADE          int64
RACA_COR           int64
ETNIA              int64
DIAS_PERM          int64
MORTE              int64
CID_MORTE         object
data_formatada    object
ano_arquivo        int64
dtype: object

Amostra:


,UF_ZI,ANO_CMPT,MES_CMPT,MUNIC_RES,MUNIC_MOV,CEP,NASC,SEXO,DT_INTER,DT_SAIDA,DIAG_PRINC,DIAG_SECUN,IDADE,COD_IDADE,RACA_COR,ETNIA,DIAS_PERM,MORTE,CID_MORTE,data_formatada,ano_arquivo
0,330455,2012,3,330350,330455,26272200,19550818,1,20120101,20120114,J189,R060,56,4,1,0,13,1,J960,2012-01-01,2012
1,330250,2012,1,330250,330250,25915000,19310415,3,20120101,20120110,J188,NaN,80,4,99,0,9,1,J188,2012-01-01,2012
2,330455,2012,3,330455,330455,20020030,20030802,1,20120101,20120103,J351,NaN,8,4,99,0,2,0,NaN,2012-01-01,2012
3,330455,2012,3,330455,330455,20720000,20031021,1,20120101,20120104,J353,NaN,8,4,3,0,3,0,NaN,2012-01-01,2012
4,330455,2012,2,330455,330455,21040300,19220324,3,20120101,20120131,J448,NaN,89,4,1,0,30,0,NaN,2012-01-01,2012
5,330000,2012,1,330475,330475,28230000,19401223,1,20120101,20120103,J189,NaN,71,4,1,0,2,0,NaN,2012-01-01,2012
6,330455,2012,1,330455,330455,22773360,19611001,1,20120101,20120110,J189,NaN,50,4,1,0,9,0,NaN,2012-01-01,2012
7,330455,2012,4,330455,330455,21070061,20061006,1,20120101,20120108,J180,NaN,5,4,3,0,7,0,NaN,2012-01-01,2012
8,330010,2012,1,330010,330010,23900620,20110129,3,20120101,20120105,J189,NaN,11,3,99,0,4,0,NaN,2012-01-01,2012
9,330455,2012,4,330455,330455,21070061,19400425,1,20120101,20120218,J180,NaN,71,4,3,0,48,1,A419,2012-01-01,2012



Valores ausentes por coluna (top 20):


,NA_count,NA_%
DIAG_SECUN,1931998,38.62
CID_MORTE,1833890,36.66
MES_CMPT,0,0.00
ANO_CMPT,0,0.00
UF_ZI,0,0.00
MUNIC_MOV,0,0.00
MUNIC_RES,0,0.00
SEXO,0,0.00
CEP,0,0.00
DT_INTER,0,0.00


## Gerar SUS para o streamlit

In [6]:
def _map_sexo_code(x) -> str:
    try:
        xi = int(x)
    except Exception:
        return str(x)
    if xi == 1:
        return "Masculino"
    if xi == 3:
        return "Feminino"
    return "Ignorado"

def _cid_to_cat3(cid: str) -> str | None:
    if isinstance(cid, str):
        cid = cid.strip().upper()
        import re
        m = re.match(r"^([A-Z]\d{2})", cid)
        return m.group(1) if m else None
    return None

def _map_grupo_J(cid: str) -> str:
    """
    Agrupa subcapítulos J00-J99 (respiratórios) a partir de DIAG_PRINC.
    """
    if not isinstance(cid, str):
        return "Não J/Indefinido"
    cid = cid.strip().upper()
    if not cid.startswith("J") or len(cid) < 3:
        return "Não J/Indefinido"
    try:
        num = int(cid[1:3])
    except Exception:
        return "J - Outros"
    if 0 <= num <= 6:
        return "J00-J06 Infecções agudas vias aéreas superiores"
    if 9 <= num <= 18:
        return "J09-J18 Influenza e Pneumonias"
    if 20 <= num <= 22:
        return "J20-J22 Outras infecções vias aéreas inferiores"
    if 40 <= num <= 47:
        return "J40-J47 Doenças crônicas das vias aéreas (ex.: DPOC, Asma)"
    if 60 <= num <= 70:
        return "J60-J70 Pneumoconioses e doenças por agentes externos"
    if 80 <= num <= 84:
        return "J80-J84 Doenças do interstício pulmonar"
    if 85 <= num <= 86:
        return "J85-J86 Supurações pulmonares"
    if 90 <= num <= 94:
        return "J90-J94 Doenças da pleura"
    if num == 95:
        return "J95 Complicações respiratórias pós-procedimentos"
    if 96 <= num <= 99:
        return "J96-J99 Outras doenças do aparelho respiratório"
    return "J - Outros"

def prepare_sus_df(df_sus: pd.DataFrame) -> pd.DataFrame:
    """
    Recebe o df_sus bruto (com nomes originais) e:
      - Garante tipos adequados (datas, numéricos)
      - Cria colunas derivadas úteis para análise (ANO, MES, ANO_MES, DIA_SEMANA, SEMANA_ANO, FAIXA_ETARIA)
      - Deriva 'CID_CAT3' e 'CID_GRUPO_J' a partir de DIAG_PRINC.
      - Cria 'SEXO_TXT' com rótulos legíveis.
    NÃO renomeia nem muda nomes originais.
    """
    df = df_sus.copy()

    if "DT_INTER" in df.columns and not pd.api.types.is_datetime64_any_dtype(df["DT_INTER"]):
        df["DT_INTER"] = pd.to_datetime(df["DT_INTER"].astype(str), format="%Y%m%d", errors="coerce")
    if "DT_SAIDA" in df.columns and not pd.api.types.is_datetime64_any_dtype(df["DT_SAIDA"]):
        df["DT_SAIDA"] = pd.to_datetime(df["DT_SAIDA"].astype(str), format="%Y%m%d", errors="coerce")

    for c in ["IDADE", "DIAS_PERM", "MORTE"]:
        if c in df.columns:
            df[c] = pd.to_numeric(df[c], errors="coerce")

    if "DT_INTER" in df.columns:
        df["ANO"] = df["DT_INTER"].dt.year
        df["MES"] = df["DT_INTER"].dt.month
        df["DIA"] = df["DT_INTER"].dt.day
        df["ANO_MES"] = df["DT_INTER"].dt.to_period("M").astype(str)
        weekday_map = {0:"segunda-feira",1:"terça-feira",2:"quarta-feira",3:"quinta-feira",4:"sexta-feira",5:"sábado",6:"domingo"}
        df["DIA_SEMANA"] = df["DT_INTER"].dt.weekday.map(weekday_map)
        df["SEMANA_ANO"] = df["DT_INTER"].dt.isocalendar().week.astype(int)

    if "IDADE" in df.columns:
        bins = [-1, 4, 14, 24, 44, 64, 79, 120]
        labels = ["0-4", "5-14", "15-24", "25-44", "45-64", "65-79", "80+"]
        df["FAIXA_ETARIA"] = pd.cut(df["IDADE"], bins=bins, labels=labels, include_lowest=True)

    if "SEXO" in df.columns:
        df["SEXO_TXT"] = df["SEXO"].apply(_map_sexo_code)

    if "DIAG_PRINC" in df.columns:
        df["DIAG_PRINC"] = df["DIAG_PRINC"].astype(str).str.upper().str.strip()
        df["CID_CAT3"] = df["DIAG_PRINC"].apply(_cid_to_cat3)
        df["CID_GRUPO_J"] = df["DIAG_PRINC"].apply(_map_grupo_J)

    
    
    return df

In [7]:
df_sus_streamlit = prepare_sus_df(df_sus)

df_sus_streamlit.shape

(5002306, 31)

In [8]:
municipios_interesse = [330455, 330510, 330320, 330285, 330045, 330350, 330170]

municipios_rio_de_janeiro = [330455]

df_sus_streamlit = df_sus_streamlit[df_sus_streamlit['MUNIC_RES'].isin(municipios_interesse)]

df_sus_streamlit.shape

(1872092, 31)

In [9]:
df_sus_streamlit["data_dia"] = pd.to_datetime(df_sus_streamlit["data_formatada"], errors="coerce").dt.normalize()

df_sus_streamlit.drop(columns=['UF_ZI', 'ANO_CMPT','DT_SAIDA', 'MES_CMPT','MUNIC_MOV', 'CEP', 'NASC','DIAG_SECUN','COD_IDADE', 'RACA_COR','ETNIA','data_formatada', 'ano_arquivo', 'SEXO'], inplace=True)

df_sus_streamlit.shape

(1872092, 18)

In [ ]:
df_sus_streamlit.dtypes

MUNIC_RES                int64
DT_INTER        datetime64[ns]
DIAG_PRINC              object
IDADE                    int64
DIAS_PERM                int64
MORTE                    int64
CID_MORTE               object
ANO                      int32
MES                      int32
DIA                      int32
ANO_MES                 object
DIA_SEMANA              object
SEMANA_ANO               int64
FAIXA_ETARIA          category
SEXO_TXT                object
CID_CAT3                object
CID_GRUPO_J             object
data_dia        datetime64[ns]
dtype: object

In [12]:
project_root = Path().resolve().parents[1]
output_dir = project_root / "data" / "DataSus" / "Streamlit"
output_dir.mkdir(parents=True, exist_ok=True)

# Divide o DataFrame em 3 partes iguais
n = len(df_sus_streamlit)
third = n // 3

df_part1 = df_sus_streamlit.iloc[:third].copy()
df_part2 = df_sus_streamlit.iloc[third:2*third].copy()
df_part3 = df_sus_streamlit.iloc[2*third:].copy()

output_csv_path1 = output_dir / "INTERNACOES_STREAMLIT_parte1.csv"
output_csv_path2 = output_dir / "INTERNACOES_STREAMLIT_parte2.csv"
output_csv_path3 = output_dir / "INTERNACOES_STREAMLIT_parte3.csv"

df_part1.to_csv(output_csv_path1, index=False)
df_part2.to_csv(output_csv_path2, index=False)
df_part3.to_csv(output_csv_path3, index=False)

print(f"Parte 1 salva em: {output_csv_path1}")
print(f"Parte 2 salva em: {output_csv_path2}")
print(f"Parte 3 salva em: {output_csv_path3}")

Parte 1 salva em: C:\Users\jhter\OneDrive - cefet-rj.br\25_2-QualiAr\data\DataSus\Streamlit\INTERNACOES_STREAMLIT_parte1.csv
Parte 2 salva em: C:\Users\jhter\OneDrive - cefet-rj.br\25_2-QualiAr\data\DataSus\Streamlit\INTERNACOES_STREAMLIT_parte2.csv
Parte 3 salva em: C:\Users\jhter\OneDrive - cefet-rj.br\25_2-QualiAr\data\DataSus\Streamlit\INTERNACOES_STREAMLIT_parte3.csv


## Filtrando dados de saúde

### Filtrando por CID-10 de Interesse

Nesta etapa, serão filtrados os códigos CID-10 relevantes para nossa análise:

**Códigos selecionados:**
- **Pneumonia:** J18, J15
- **Insuficiência Respiratória:** J21, J96  
- **Bronquites:** J40, J41, J42
- **Enfisema/DPOC:** J43, J44
- **Asma:** J45, J46

In [16]:
cids_associados_a_poluicao = {
    "J18", "J15", # pneumonia
    "J21", "J96", # insuficiência respiratória
    "J40", "J41", "J42",  # bronquites
    "J43", "J44",         # enfisema / DPOC
    "J45", "J46"          # asma / status asmático
}

col_diag = "DIAG_PRINC"  
df_sus[col_diag] = df_sus[col_diag].astype(str).str.upper().str.strip()

# Remove caracteres não alfanuméricos 
df_sus[col_diag] = df_sus[col_diag].str.replace(r"[^A-Z0-9]", "", regex=True)

# Prefixo CID: 3 primeiros caracteres
df_sus["CID_PREFIXO"] = df_sus[col_diag].str[:3]

sus_filtrado = df_sus[df_sus["CID_PREFIXO"].isin(cids_associados_a_poluicao)].copy()

print(sus_filtrado.shape)
print(sus_filtrado["CID_PREFIXO"].value_counts())
display(sus_filtrado.head())

(3735124, 22)
CID_PREFIXO
J18    1548123
J15     972265
J21     278527
J45     252839
J96     241690
J44     215236
J40     103721
J46      48549
J43      41336
J41      24823
J42       8015
Name: count, dtype: int64


,UF_ZI,ANO_CMPT,MES_CMPT,MUNIC_RES,MUNIC_MOV,CEP,NASC,SEXO,DT_INTER,DT_SAIDA,DIAG_PRINC,DIAG_SECUN,IDADE,COD_IDADE,RACA_COR,ETNIA,DIAS_PERM,MORTE,CID_MORTE,data_formatada,ano_arquivo,CID_PREFIXO
0,330455,2012,3,330350,330455,26272200,19550818,1,20120101,20120114,J189,R060,56,4,1,0,13,1,J960,2012-01-01,2012,J18
1,330250,2012,1,330250,330250,25915000,19310415,3,20120101,20120110,J188,NaN,80,4,99,0,9,1,J188,2012-01-01,2012,J18
4,330455,2012,2,330455,330455,21040300,19220324,3,20120101,20120131,J448,NaN,89,4,1,0,30,0,NaN,2012-01-01,2012,J44
5,330000,2012,1,330475,330475,28230000,19401223,1,20120101,20120103,J189,NaN,71,4,1,0,2,0,NaN,2012-01-01,2012,J18
6,330455,2012,1,330455,330455,22773360,19611001,1,20120101,20120110,J189,NaN,50,4,1,0,9,0,NaN,2012-01-01,2012,J18


### Filtrando por Municípios de Interesse

Nesta etapa, serão filtrados os municípios relevantes para nossa análise:

#### Municípios Selecionados

- **330455** - Rio de Janeiro
- **330510** - São João de Meriti
- **330320** - Nilópolis
- **330285** - Mesquita
- **330045** - Belford Roxo
- **330350** - Nova Iguaçu
- **330170** - Duque de Caxias

In [17]:
municipios_interesse = [330455,    330510,     330320,     330285,    330045,        330350,        330170]

municipios_rio_de_janeiro = [330455]

sus_filtrado = sus_filtrado[sus_filtrado['MUNIC_RES'].isin(municipios_interesse)]

print("Dimensões após filtro por municípios:", sus_filtrado.shape)

Dimensões após filtro por municípios: (1435466, 22)


### Filtrando por dias de permanência maior que zero

In [18]:
sus_filtrado = sus_filtrado[sus_filtrado['DIAS_PERM'] > 0]
print("Dimensões após filtro por permanência:", sus_filtrado.shape)

Dimensões após filtro por permanência: (1412369, 22)



## Série diária de internações


### Corrigindo data

In [19]:
sus_filtrado = sus_filtrado.copy()
sus_filtrado["data_dia"] = pd.to_datetime(sus_filtrado["data_formatada"], errors="coerce").dt.normalize()
sus_filtrado = sus_filtrado.dropna(subset=["data_dia"])

### Gerando quantidade de internações por dia

In [20]:
internacoes_diario = (
    sus_filtrado.groupby("data_dia")
    .size()
    .rename("num_internacoes")
    .astype("int64")
    .reset_index()
    .sort_values("data_dia")
)

### Preenchendo calendário
Preencher calendário (datas faltantes -> 0 internações)

In [21]:
idx_full = pd.date_range(internacoes_diario["data_dia"].min(),
                         internacoes_diario["data_dia"].max(),
                         freq="D")

internacoes_rj = (
    internacoes_diario.set_index("data_dia")
    .reindex(idx_full, fill_value=0)
    .rename_axis("data_dia")
    .reset_index()
)

### Gerando CSV de saída

In [26]:
project_root = Path().resolve().parents[1]  
output_dir = project_root / "data" / "DataSus"
output_dir.mkdir(parents=True, exist_ok=True)

output_csv_path = output_dir / "INTERNACOES_DOENCA_RESP_RJ.csv"
internacoes_rj.to_csv(output_csv_path, index=False, encoding="utf-8")

# Contagem de dias com 0 internações
qtd_zeros = int((internacoes_rj["num_internacoes"] == 0).sum())
print(f"Arquivo salvo em: {output_csv_path}")
print(f"Dias com 0 internações: {qtd_zeros}")
print(internacoes_rj.head())

Arquivo salvo em: C:\Users\jhter\OneDrive - cefet-rj.br\25_2-QualiAr\data\DataSus\INTERNACOES_DOENCA_RESP_RJ.csv
Dias com 0 internações: 0
    data_dia  num_internacoes
0 2012-01-01              507
1 2012-01-02              728
2 2012-01-03              429
3 2012-01-04              416
4 2012-01-05              403


In [27]:
display(internacoes_rj[["data_dia", "num_internacoes"]].head(10))

,data_dia,num_internacoes
0,2012-01-01,507
1,2012-01-02,728
2,2012-01-03,429
3,2012-01-04,416
4,2012-01-05,403
5,2012-01-06,520
6,2012-01-07,299
7,2012-01-08,390
8,2012-01-09,442
9,2012-01-10,546
